In [1]:
from time import sleep
from os import path
import googleapiclient.discovery
import json
import os

In [1]:
CHANNEL_YUTT = 'UCQJ6ESCWQotBwtJm0Ff_gyQ'
DEVELOPER_KEY = 'KEY'

In [3]:
def yt_search_list(channelId, **kwargs):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.search().list(
        part="snippet",
        channelId=channelId,
        order='date',
        maxResults=50,
        **kwargs
    )
    response = request.execute()

    return response

In [4]:
def yt_videos_list(ids):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.videos().list(
        part="snippet,contentDetails,topicDetails,statistics,liveStreamingDetails",
        maxResults=50,
        id=ids
    )
    response = request.execute()
    
    return response

In [5]:
with open('./videos_list_yutt.json') as f:
    videos_list_yutt = sorted(json.loads(f.read()), key=lambda x: x['snippet']['publishedAt'], reverse=True)

In [6]:
new_videos = yt_search_list(CHANNEL_YUTT, publishedAfter=videos_list_yutt[0]['snippet']['publishedAt'])
assert new_videos['pageInfo']['totalResults'] <= 50

In [7]:
new_videos

{'kind': 'youtube#searchListResponse',
 'etag': 'x7LTqatx8ZVHKB72hs3QSDucdD8',
 'regionCode': 'DE',
 'pageInfo': {'totalResults': 8, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'wvRSBb5FGowdBQTMuDESN4802lE',
   'id': {'kind': 'youtube#video', 'videoId': 'ksYXlf7Nbgw'},
   'snippet': {'publishedAt': '2020-07-12T08:20:38Z',
    'channelId': 'UCQJ6ESCWQotBwtJm0Ff_gyQ',
    'title': 'Meditation Q&amp;A, July 11, 2020',
    'description': 'How To Meditate Booklet: https://htm.sirimangalo.org/ To join our meditation community and sign up for an at-home meditation course, please visit ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ksYXlf7Nbgw/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/ksYXlf7Nbgw/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/ksYXlf7Nbgw/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelT

In [8]:
video_ids = set([v['id'] for v in videos_list_yutt if v['kind'] == 'youtube#video'])
new_video_ids = [v['id']['videoId'] for v in new_videos['items'] if not (v['id']['kind'] == 'youtube#video' and v['id']['videoId'] in video_ids)]

In [9]:
new_video_ids

['ksYXlf7Nbgw',
 'Jhfq_dx1A00',
 'ZWJVvsNzA8Y',
 'f6X13Ar9Cfw',
 'W6wHkfolnhc',
 '-dULR0idhG0',
 'xBAUrAQhPYY']

In [10]:
new_video_list = yt_videos_list(','.join(new_video_ids))
videos_list_yutt_updated = new_video_list['items'] + videos_list_yutt
assert set([v['id'] for v in videos_list_yutt_updated if v['kind'] == 'youtube#video']) == (set(new_video_ids).union(video_ids))

In [11]:
len(videos_list_yutt)

1722

In [12]:
len(videos_list_yutt_updated)

1729

In [13]:
with open('./videos_list_yutt.json', 'w') as f:
    f.write(json.dumps(videos_list_yutt_updated, indent=2))